In [1]:
from pathlib import Path
from datetime import timedelta
import re

import pandas as pd

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.table import Table, TableStyleInfo


# =====================================================
# RUTAS Y ENTRADAS/SALIDAS
# =====================================================
# Estructura asumida:
# - Repositorio1/
#   - notebooks/   (este notebook)
#   - outputs/     (salidas del pipeline)

NOTEBOOK_DIR = Path.cwd()
REPO_ROOT = NOTEBOOK_DIR.parent

LOCAL_INPUT = REPO_ROOT / "outputs" / "validations_clean.csv"
OUTPUT_XLSX = REPO_ROOT / "outputs" / "viajes_generados.xlsx"

print("Leyendo data desde:", LOCAL_INPUT)

if not LOCAL_INPUT.exists():
    raise FileNotFoundError(
        f"No se encontró el archivo local esperado:\n{LOCAL_INPUT}"
    )


# =====================================================
# PARÁMETROS DE NEGOCIO
# =====================================================
# UMBRAL_MAX: ventana máxima permitida entre ENTRY y EXIT para considerar un viaje completo.
UMBRAL_MAX = timedelta(minutes=25)

Leyendo data desde: C:\Users\DiegoEveraldoFernand\Python\Repositorio1\outputs\validations_clean.csv


In [2]:
# =====================================================
# CARGA Y NORMALIZACIÓN MÍNIMA
# =====================================================
df = pd.read_csv(LOCAL_INPUT, dtype=str)

required_cols = {
    "DateTime",
    "SupportId",
    "StopPlaceShortName",
    "TransactionType",
    "ProfileName",
    "EquipmentModel",
}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns: {sorted(missing)}")

df["DateTime"] = pd.to_datetime(df["DateTime"], errors="coerce")
df = df[df["DateTime"].notna()].copy()

for c in ["SupportId", "StopPlaceShortName", "TransactionType", "ProfileName", "EquipmentModel"]:
    df[c] = df[c].astype(str).str.strip()

df["TransactionType"] = df["TransactionType"].str.upper()

# Orden determinista por tarjeta y fecha-hora (base para el emparejamiento)
df = df.sort_values(["SupportId", "DateTime"]).reset_index(drop=True)

# Campos auxiliares para reporte
df["Fecha"] = df["DateTime"].dt.date.astype(str)
df["Hora"] = df["DateTime"].dt.strftime("%H:%M:%S")

print("Rows loaded:", f"{len(df):,}")

Rows loaded: 14,224


In [3]:
# =====================================================
# CODIFICACIÓN DE ESTACIONES
# =====================================================
# Ejemplo de anonimización
stations = sorted(df["StopPlaceShortName"].dropna().unique())
station_map = {name: f"ST_{i+1:02d}" for i, name in enumerate(stations)}
df["StationCode"] = df["StopPlaceShortName"].map(station_map)


# =====================================================
# CONSTRUCCIÓN DE VIAJES (EMPAREJAMIENTO 1:1 ENTRY→EXIT)
# =====================================================
def construir_viajes(df_total: pd.DataFrame, umbral: timedelta) -> pd.DataFrame:
    """
    Construye viajes por SupportId con una regla operativa clara:

    - Cada ENTRY inicia un viaje.
    - Se busca el primer EXIT posterior que ocurra dentro del umbral.
    - Cada EXIT se utiliza una sola vez (se consume), evitando reasignaciones.
    - Si no hay EXIT válido dentro del umbral, el viaje queda como incompleto.

    """
    viajes = []

    for support_id, g in df_total.groupby("SupportId", sort=False):
        # Orden estable dentro del grupo (mergesort conserva orden relativo si hay empates)
        g = g.sort_values("DateTime", kind="mergesort").copy()

        entradas = g[g["TransactionType"] == "ENTRY"].copy()
        salidas = g[g["TransactionType"] == "EXIT"].copy()

        if entradas.empty:
            continue

        salidas_idx = list(salidas.index)
        sal_pos = 0

        for ent in entradas.itertuples(index=False):
            ent_time = ent.DateTime
            matched_idx = None

            # Avanza el puntero para ignorar salidas <= ENTRY (no pueden cerrar este viaje)
            while (
                sal_pos < len(salidas_idx)
                and salidas.loc[salidas_idx[sal_pos], "DateTime"] <= ent_time
            ):
                sal_pos += 1

            j = sal_pos
            while j < len(salidas_idx):
                idx_sal = salidas_idx[j]
                sal_time = salidas.loc[idx_sal, "DateTime"]
                delta = sal_time - ent_time

                if delta < timedelta(0):
                    j += 1
                    continue

                # Match cuando cae dentro de la ventana operativa
                if delta <= umbral:
                    matched_idx = idx_sal
                    break

                # Si ya excede umbral, no habrá match para este ENTRY (por orden temporal)
                break

            if matched_idx is not None:
                sal = salidas.loc[matched_idx]
                exit_time = sal["DateTime"]

                viajes.append({
                    "SupportId": support_id,
                    "ProfileName": ent.ProfileName,
                    "Fecha": ent.Fecha,
                    "Est_Ingreso": ent.StationCode,
                    "Hora_Ingreso": ent.Hora,
                    "Est_Salida": sal["StationCode"],
                    "Hora_Salida": exit_time.strftime("%H:%M:%S"),
                    "Tiempo_min": round((exit_time - ent_time).total_seconds() / 60, 2),
                    "DateTime_Ingreso": ent_time,
                })

                # Consumir la salida encontrada: no puede cerrar otro ENTRY
                salidas_idx.pop(j)
            else:
                # Viaje incompleto: útil para control operativo y análisis de anomalías
                viajes.append({
                    "SupportId": support_id,
                    "ProfileName": ent.ProfileName,
                    "Fecha": ent.Fecha,
                    "Est_Ingreso": ent.StationCode,
                    "Hora_Ingreso": ent.Hora,
                    "Est_Salida": None,
                    "Hora_Salida": None,
                    "Tiempo_min": None,
                    "DateTime_Ingreso": ent_time,
                })

    df_viajes = pd.DataFrame(viajes)

    if df_viajes.empty:
        return df_viajes

    # Etiqueta de ruta solo para viajes completos
    mask = df_viajes["Est_Salida"].notna()
    df_viajes["Ruta"] = None
    df_viajes.loc[mask, "Ruta"] = (
        df_viajes.loc[mask, "Est_Ingreso"] + " a " + df_viajes.loc[mask, "Est_Salida"]
    )
    df_viajes["Viaje"] = df_viajes["Ruta"].where(
        df_viajes["Ruta"].notna(),
        "No completó el viaje"
    )

    return df_viajes.sort_values(
        ["SupportId", "DateTime_Ingreso"]
    ).reset_index(drop=True)


df_viajes = construir_viajes(df, UMBRAL_MAX)

if df_viajes.empty:
    raise RuntimeError("No trips were generated.")

print("Trips generated:", f"{len(df_viajes):,}")

Trips generated: 7,123


In [4]:
# =====================================================
# EXPORTACIÓN A EXCEL
# =====================================================
# Se exportan campos orientados a auditoría operativa y análisis:
# - Identificación (SupportId, ProfileName)
# - Resultado del viaje (Viaje, ruta, estaciones, horas, duración)
cols_export = [
    "SupportId",
    "ProfileName",
    "Viaje",
    "Fecha",
    "Est_Ingreso",
    "Hora_Ingreso",
    "Est_Salida",
    "Hora_Salida",
    "Tiempo_min",
]

df_export = df_viajes.copy()
dt_ing = pd.to_datetime(df_export["DateTime_Ingreso"], errors="coerce")
df_export["Periodo"] = dt_ing.dt.to_period("M").astype(str)

wb = Workbook()
wb.remove(wb.active)


def sanitize_name(name: str) -> str:
    """Asegura nombres compatibles con Excel (máx 31 caracteres y sin símbolos no permitidos)."""
    name = re.sub(r"[^A-Za-z0-9_]", "_", str(name))
    return name[:31] if name else "Sheet"


for periodo, df_p in df_export.groupby("Periodo", sort=False):
    ws = wb.create_sheet(title=sanitize_name(periodo))
    df_p = df_p[cols_export]

    for row in dataframe_to_rows(df_p, index=False, header=True):
        ws.append(row)

    ref = f"A1:{get_column_letter(ws.max_column)}{ws.max_row}"
    table = Table(
        displayName=sanitize_name(f"tbl_{periodo}"),
        ref=ref
    )
    table.tableStyleInfo = TableStyleInfo(
        name="TableStyleMedium9",
        showRowStripes=True
    )
    ws.add_table(table)

    # Ajuste simple de ancho de columnas para lectura
    for col_cells in ws.columns:
        width = max(len(str(c.value)) if c.value else 0 for c in col_cells) + 2
        ws.column_dimensions[col_cells[0].column_letter].width = min(width, 45)

wb.save(OUTPUT_XLSX)

print("Excel generado en:", OUTPUT_XLSX.resolve())
print("Total de viajes exportados:", f"{len(df_export):,}")

Excel generado en: C:\Users\DiegoEveraldoFernand\Python\Repositorio1\outputs\viajes_generados.xlsx
Total de viajes exportados: 7,123
